Code for data retrieval from TMDb API was adapted and modified from Sohier Dane's code
https://gist.github.com/SohierDane/4a84cb96d220fc4791f52562be37968b

In [2]:




import gzip
import json
import os
import pandas as pd
import requests

from io import BytesIO
from time import sleep


BASE_API_CALL = 'https://api.themoviedb.org/3/{category}/{entry_id}?api_key={api_key}{category_specifics}'
CATEGORIES = ['movie']
DOWNLOADS_PER_DISK_WRITE = 40
MAX_DOWNLOADS_PER_SECOND = 4
MAX_ATTEMPTS = 3
RATE_LIMITER_DELAY_SECONDS = 10
RATE_LIMIT_EXCEEDED_STATUS_CODE = 429
SUCCESSFUL_CALL_STATUS_CODE = 200

CATEGORY_SPECIFIC_CALLS = {
    'movie': '&append_to_response=credits,keywords',
                            }

JSON_COLUMNS = {
    'genres',
    'keywords',
    'production_countries',
    'production_companies',
    'spoken_languages'
                }

KEYS_TO_DROP = {
    'adult',
    'backdrop_path',
    'belongs_to_collection',
    'imdb_id',
    'poster_path',
    'profile_path',
    'video',
    'homepage',
    'credits'
                }


def was_successful(response):
    return response.status_code == SUCCESSFUL_CALL_STATUS_CODE


def was_rate_limited(response):
    return response.status_code == RATE_LIMIT_EXCEEDED_STATUS_CODE


def make_request(call_url, prior_attempts=0):
    if prior_attempts >= MAX_ATTEMPTS:
        return None
    response = requests.get(call_url)
    if was_rate_limited(response):
        sleep(RATE_LIMITER_DELAY_SECONDS)
    sleep(1 / MAX_DOWNLOADS_PER_SECOND)
    if was_successful(response):
        return response.json()
    else:
        sleep(1)  # attempt to sleep through any intermittent issues
        return make_request(call_url, prior_attempts + 1)


def make_detail_request(category, entry_id):
    category_specifics = ''
    if category in CATEGORY_SPECIFIC_CALLS:
        category_specifics = CATEGORY_SPECIFIC_CALLS[category]
    call_url = BASE_API_CALL.format(
        category=category,
        entry_id=entry_id,
        api_key=API_KEY,
        category_specifics=category_specifics,
                                    )
    return make_request(call_url)


def load_api_key():
    return 'bf0cb24a4dbe7d1eb14b726c1313cbf5'


def make_category_id_url_suffix(category, extension='json'):
    year = str(pd.datetime.today().year)
    month = str(pd.datetime.today().month).zfill(2)
    day = str(pd.datetime.today().day - 1).zfill(2)
    return '_'.join([category, 'ids', month, day, year]) + '.' + extension


def download_id_list_as_csv(category):
    # see daily file export list docs at:
    # https://developers.themoviedb.org/3/getting-started/daily-file-exports
    print(f'Downloading list of ids for {category}')
    id_list_name = make_category_id_url_suffix(category)
    ID_LISTS_RAW_URL = 'http://files.tmdb.org/p/exports/{0}.gz'.format(id_list_name)
    with gzip.open(BytesIO(requests.get(ID_LISTS_RAW_URL).content), 'r') as f_open:
        id_list = f_open.readlines() #.decode('utf-8')
    # original 'json' is malformed, is actually one dict per line
    ids = pd.DataFrame([json.loads(x) for x in id_list])
    # some entries in the movie id list appear to be collections rather than movies
    if 'original_title' in ids.columns:
        ids.original_title = ids.original_title.apply(str)
        ids = ids[~ids.original_title.str.endswith(' Collection')].copy()
    # Drop adult films
    if 'adult' in ids.columns:
        ids = ids[~ids['adult']].copy()
    ids.to_csv(category + '_ids.csv', index=False)


def load_id_list(category):
    if not os.path.exists(category + '_ids.csv'):
        download_id_list_as_csv(category)
    df = pd.read_csv(category + '_ids.csv')
    return df.id.values.tolist()

def export_data(category, all_entries):
    if not all_entries:
        return None
    df = pd.DataFrame(all_entries)
    df = df[[x for x in df.columns if x not in KEYS_TO_DROP]].copy()
    
    # Drop those with no id
    if len(df[df.id.isnull()]) > 0:
        print(f'Dropping {len(df[df.id.isnull()])} entries without ids')
        df = df[~df.id.isnull()]
    # Download English movies only
    df = df.loc[(df['original_language']=='en')]
    
    
    df = df[df.id.apply(lambda x: str(x).isnumeric())]
    df['keywords'] = df['keywords'].apply(lambda x:
        x['keywords'] if 'keywords' in x else [])
    for column in JSON_COLUMNS:
        df[column] = df[column].apply(json.dumps)
    needs_header = not(os.path.exists(category + '_data.csv'))
    df.to_csv(category + '_data.csv', index=False, mode='a+', header=needs_header)


def download_ids(category, id_list):
    if os.path.exists(category + '_data.csv'):
        existing_ids = pd.read_csv(category + '_data.csv', usecols=['id'], dtype=object)
        set(existing_ids.id.values.tolist())
        id_list = [x for x in id_list if str(x) not in existing_ids]
    counter = 0
    all_entries = []
    print(f'Downloading details for {category}')
    for movie_id in id_list:
        current_data = make_detail_request(category, movie_id)
        if not current_data:
            print(f'Failed on id # {movie_id}')
            continue
        counter += 1
        all_entries.append(current_data)
        if counter % DOWNLOADS_PER_DISK_WRITE == 0:
            print(f'Finished downloading {counter} entries for {category}')
            export_data(category, all_entries)
            all_entries = []
    export_data(category, all_entries)


def download_all_data():
    for category in CATEGORIES:
        download_ids(category, load_id_list(category))


if __name__ == '__main__':
    API_KEY = load_api_key()
    download_all_data()



Finished downloading 40 entries for movie
Finished downloading 80 entries for movie
Finished downloading 120 entries for movie
Finished downloading 160 entries for movie
Finished downloading 200 entries for movie
Finished downloading 240 entries for movie
Finished downloading 280 entries for movie
Finished downloading 320 entries for movie
Finished downloading 360 entries for movie
Finished downloading 400 entries for movie
Finished downloading 440 entries for movie
Finished downloading 480 entries for movie
Finished downloading 520 entries for movie
Finished downloading 560 entries for movie
Finished downloading 600 entries for movie
Finished downloading 640 entries for movie
Finished downloading 680 entries for movie
Finished downloading 720 entries for movie
Finished downloading 760 entries for movie
Finished downloading 800 entries for movie
Finished downloading 840 entries for movie
Finished downloading 880 entries for movie
Finished downloading 920 entries for movie
Finished down

Finished downloading 7520 entries for movie
Finished downloading 7560 entries for movie
Finished downloading 7600 entries for movie
Finished downloading 7640 entries for movie
Finished downloading 7680 entries for movie
Finished downloading 7720 entries for movie
Finished downloading 7760 entries for movie
Finished downloading 7800 entries for movie
Finished downloading 7840 entries for movie
Finished downloading 7880 entries for movie
Finished downloading 7920 entries for movie
Finished downloading 7960 entries for movie
Finished downloading 8000 entries for movie
Finished downloading 8040 entries for movie
Finished downloading 8080 entries for movie
Finished downloading 8120 entries for movie
Finished downloading 8160 entries for movie
Finished downloading 8200 entries for movie
Finished downloading 8240 entries for movie
Finished downloading 8280 entries for movie
Finished downloading 8320 entries for movie
Finished downloading 8360 entries for movie
Finished downloading 8400 entrie

Finished downloading 14840 entries for movie
Finished downloading 14880 entries for movie
Finished downloading 14920 entries for movie
Finished downloading 14960 entries for movie
Finished downloading 15000 entries for movie
Finished downloading 15040 entries for movie
Finished downloading 15080 entries for movie
Finished downloading 15120 entries for movie
Finished downloading 15160 entries for movie
Finished downloading 15200 entries for movie
Finished downloading 15240 entries for movie
Finished downloading 15280 entries for movie
Finished downloading 15320 entries for movie
Finished downloading 15360 entries for movie
Finished downloading 15400 entries for movie
Finished downloading 15440 entries for movie
Finished downloading 15480 entries for movie
Finished downloading 15520 entries for movie
Finished downloading 15560 entries for movie
Finished downloading 15600 entries for movie
Finished downloading 15640 entries for movie
Finished downloading 15680 entries for movie
Finished d

Finished downloading 22160 entries for movie
Finished downloading 22200 entries for movie
Finished downloading 22240 entries for movie
Finished downloading 22280 entries for movie
Finished downloading 22320 entries for movie
Finished downloading 22360 entries for movie
Finished downloading 22400 entries for movie
Finished downloading 22440 entries for movie
Finished downloading 22480 entries for movie
Finished downloading 22520 entries for movie
Finished downloading 22560 entries for movie
Finished downloading 22600 entries for movie
Finished downloading 22640 entries for movie
Finished downloading 22680 entries for movie
Finished downloading 22720 entries for movie
Finished downloading 22760 entries for movie
Finished downloading 22800 entries for movie
Finished downloading 22840 entries for movie
Finished downloading 22880 entries for movie
Finished downloading 22920 entries for movie
Finished downloading 22960 entries for movie
Finished downloading 23000 entries for movie
Finished d

Finished downloading 29440 entries for movie
Finished downloading 29480 entries for movie
Finished downloading 29520 entries for movie
Finished downloading 29560 entries for movie
Finished downloading 29600 entries for movie
Finished downloading 29640 entries for movie
Finished downloading 29680 entries for movie
Finished downloading 29720 entries for movie
Finished downloading 29760 entries for movie
Finished downloading 29800 entries for movie
Finished downloading 29840 entries for movie
Finished downloading 29880 entries for movie
Finished downloading 29920 entries for movie
Finished downloading 29960 entries for movie
Finished downloading 30000 entries for movie
Finished downloading 30040 entries for movie
Finished downloading 30080 entries for movie
Finished downloading 30120 entries for movie
Finished downloading 30160 entries for movie
Finished downloading 30200 entries for movie
Finished downloading 30240 entries for movie
Finished downloading 30280 entries for movie
Finished d

Finished downloading 36760 entries for movie
Finished downloading 36800 entries for movie
Finished downloading 36840 entries for movie
Finished downloading 36880 entries for movie
Finished downloading 36920 entries for movie
Finished downloading 36960 entries for movie
Finished downloading 37000 entries for movie
Finished downloading 37040 entries for movie
Finished downloading 37080 entries for movie
Finished downloading 37120 entries for movie
Finished downloading 37160 entries for movie
Finished downloading 37200 entries for movie
Finished downloading 37240 entries for movie
Finished downloading 37280 entries for movie
Finished downloading 37320 entries for movie
Finished downloading 37360 entries for movie
Finished downloading 37400 entries for movie
Finished downloading 37440 entries for movie
Finished downloading 37480 entries for movie
Finished downloading 37520 entries for movie
Finished downloading 37560 entries for movie
Finished downloading 37600 entries for movie
Finished d

KeyboardInterrupt: 